TODO

- Dataset, DataLoaderを使った形に書き換える

### Libraries

In [1]:
from typing import Union, Optional, Any, Literal, TypeVar, Generic
import sys
from dataclasses import dataclass
import json
import math
import os
import pickle
from copy import deepcopy

if sys.version_info >= (3, 9):
    from collections.abc import Sequence, Callable
else:
    from typing import Sequence, Callable

import pandas as pd
import numpy as np
from scipy.constants import zero_Celsius
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import GroupKFold, KFold, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, PowerTransformer
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import (
    mean_absolute_error,
    r2_score,
    mean_squared_error,
    max_error,
)
from sklearn.metrics import (
    f1_score,
    recall_score,
    precision_score,
    accuracy_score,
)
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.utils
import torch.utils.data
from torch import optim
from torch.optim.lr_scheduler import StepLR
from tqdm.auto import tqdm

In [2]:
T = TypeVar("T")

### User parameters

recorded the actual values when training the model for the paper

In [3]:
COLNAME_SOLUBILITY_POLYMER_PAIR = "ps_pair"

In [4]:
# device
if torch.cuda.is_available():
    device = "cuda:0"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

IDX_CV_TEST = 1
"""index of the set of randomly split test data stored in test_cv_idx.pkl"""

# data splitting
test_ratio = 0.2
"""ratio of total data in computational Chi data and solubility data used for test data"""
n_CV_val = 5
"""k-fold cross-validation for hyperparameter tuning"""
test_ratio_final = 0.2
"""ratio of training data used for validation during final training after selection of the best hyperparameters"""

# hyperparameters
n_hpara = 100
"""number of samples in the hyperparameter space"""
learning_rates = [0.001, 0.01]
"""range for learning rates"""
alpha1s = (0.0, 1.0)
"""range for lambda_c (can be set to zero only for 2-task model excluding PoLyInfo dataset)"""
alpha2s = (0.0, 1.0)
"""range for lambda_s (can be set to zero only for 2-task model excluding COSMO-RS dataset)"""
dim_outs = (3, 40)
"""range for dimension of Z"""

# model training
dir_base = f"hyper_groupCV/MT_testset_{IDX_CV_TEST}"
"""output directory for hyperparameter selection"""
n_final_model = 10
"""number of retrained model ensembles after selection of the best hyperparameters"""

n_NNlayer = 3
"""number of layer in the sub-network of fully connection MLPs"""
sch_step_size = 10
"""step size for learning rate scheduler"""
sch_gamma = 0.5
"""gamma parameter for learning rate scheduler"""
epochs_s = 0
"""number of pre-training steps based on solubility data (not used in the paper)"""
epochs = 50
"""number of max. epoch for the main training"""
burn_in = 1
"""burn in epoch (skip this number of epoch when selecting the epoch with the lowest validation loss)"""

n_minibatch_PI = 20
"""number of minibatch for solubility data"""
n_minibatch_COSMO = 10
"""number of minibatch for computational Chi data"""
n_minibatch_CHI = 5
"""number minibatch for experimental Chi data"""
n_factor_CHI = int(n_minibatch_PI / n_minibatch_CHI)
n_factor_COSMO = int(n_minibatch_PI / n_minibatch_COSMO)

# other internal parameters
temp_dim = 1
"""take 1 or 2 only. 1: linear temperature dependence only; 2: 1/T^2 term included"""
loss_factor_target = 1
"""multiplier to adjust target loss contribution to total loss for model training"""
no_target_BN = True
"""True: do not include experimental Chi data for batch normalization in training"""
no_COSMO_BN = True
"""True: do not include COSMO data for batch normalization in training"""

seed_hyper = 2022
"""random seed for hyperparameter samples"""

rng = np.random.default_rng(seed_hyper)

In [5]:
# define data container
@dataclass
class TargetContainer(Generic[T]):
    PI: Optional[T] = None
    COSMO: Optional[T] = None
    Chi: Optional[T] = None

    def keys(self) -> tuple[Literal["PI", "COSMO", "Chi"], ...]:
        return tuple(
            _key
            for _key in ("PI", "COSMO", "Chi")
            if getattr(self, _key) is not None
        )

    def __getitem__(self, _key: Literal["PI", "COSMO", "Chi"]) -> T:
        if _key in ("PI", "COSMO", "Chi"):
            return getattr(self, _key)
        else:
            raise NotImplementedError

## Load data

In [6]:
DIRPATH_LOAD = "./sample_data"

### Data

In [7]:
data_container = TargetContainer(
    PI=pd.read_csv(f"{DIRPATH_LOAD}/data_PI.csv", index_col=0)
)
print(data_container.PI.shape)
data_container.PI.head()

(1190, 3)


,ps_pair,polymer_class,soluble
0,[H]/C1=C(/Cl)C([H])([H])C([H])([H])/C([H])=C(\...,polychlorox,True
1,[H]/C1=C(/Cl)C([H])([H])C([H])([H])/C([H])=C(\...,polychlorox,True
2,[H]/C1=C(/Cl)C([H])([H])C([H])([H])/C([H])=C(\...,polychlorox,True
3,[H]/C1=C(/Cl)C([H])([H])C([H])([H])/C([H])=C(\...,polychlorox,True
4,[H]/C1=C(/Cl)C([H])([H])C([H])([H])/C([H])=C(\...,polychlorox,True


In [8]:
data_container.COSMO = pd.read_csv(
    f"{DIRPATH_LOAD}/data_COSMO.csv", index_col=0
)
print(data_container.COSMO.shape)
data_container.COSMO.head()

(1206, 5)


,ps_pair,temp,chi,polymer_class,ps_pair_nonCyclic
0,[H]/C1=C(/Cl)C([H])([H])C([H])([H])/C([H])=C(\...,100.0,0.358063,cellulose,NaN
1,[H]/C1=C(/Cl)C([H])([H])C([H])([H])/C([H])=C(\...,100.0,0.358888,cellulose,NaN
2,[H]/C1=C(/Cl)C([H])([H])C([H])([H])/C([H])=C(\...,100.0,0.327410,cellulose,NaN
3,[H]/C1=C(/Cl)C([H])([H])C([H])([H])/C([H])=C(\...,100.0,0.604347,cellulose,NaN
4,[H]/C1=C(/Cl)C([H])([H])C([H])([H])/C([H])=C(\...,100.0,0.556457,cellulose,NaN


In [9]:
data_container.Chi = pd.read_csv(f"{DIRPATH_LOAD}/data_Chi.csv", index_col=0)
print(data_container.Chi.shape)
data_container.Chi.head()

(1190, 8)


,ps_pair,ps_pair_nonCyclic,temp,chi,polymer_class,DIFF,SIGN2,SIGN3
0,[H]/C1=C(/Cl)C([H])([H])C([H])([H])/C([H])=C(\...,*C/C=C(\Cl)C*_ClC(Cl)(Cl)Cl,100.0,0.23,polychlorox,NaN,NaN,NaN
1,[H]/C1=C(/Cl)C([H])([H])C([H])([H])/C([H])=C(\...,*C/C=C(\Cl)C*_[H]C(Cl)(Cl)Cl,100.0,0.28,polychlorox,NaN,NaN,NaN
2,[H]/C1=C(/Cl)C([H])([H])C([H])([H])/C([H])=C(\...,*C/C=C(\Cl)C*_[H]C(Cl)=C(Cl)Cl,100.0,0.24,polychlorox,NaN,NaN,NaN
3,[H]/C1=C(/Cl)C([H])([H])C([H])([H])/C([H])=C(\...,*C/C=C(\Cl)C*_[H]C([H])(Cl)C([H])([H])Cl,100.0,0.48,polychlorox,NaN,NaN,NaN
4,[H]/C1=C(/Cl)C([H])([H])C([H])([H])/C([H])=C(\...,*C/C=C(\Cl)C*_[H]C([H])(Cl)Cl,100.0,0.43,polychlorox,NaN,NaN,NaN


### Descriptor

maybe calculate by `sample_code_for_descriptor_calculation.ipynb`

In [10]:
desc_container = TargetContainer(
    PI=pd.read_csv(f"{DIRPATH_LOAD}/desc_PI.csv", index_col=0)
)
print(desc_container.PI.shape)
desc_container.PI.head()

(1190, 794)


,Polymer_mass_H,Polymer_mass_C,Polymer_mass_N,Polymer_mass_O,Polymer_mass_F,Polymer_mass_P,Polymer_mass_S,Polymer_mass_Cl,Polymer_mass_Br,Polymer_mass_I,...,Solvent_fr_sulfide,Solvent_fr_sulfonamd,Solvent_fr_sulfone,Solvent_fr_term_acetylene,Solvent_fr_tetrazole,Solvent_fr_thiazole,Solvent_fr_thiocyan,Solvent_fr_thiophene,Solvent_fr_unbrch_alkane,Solvent_fr_urea
0,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,0,0,0,0,0,0,0,0,0,0
1,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,0,0,0,0,0,0,0,0,0,0
2,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,0,0,0,0,0,0,0,0,0,0
3,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,0,0,0,0,0,0,0,0,0,0
4,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,0,0,0,0,0,0,0,0,0,0


In [11]:
desc_container.COSMO = pd.read_csv(
    f"{DIRPATH_LOAD}/desc_COSMO.csv", index_col=0
)
print(desc_container.COSMO.shape)
desc_container.COSMO.head()

(1206, 794)


,Polymer_mass_H,Polymer_mass_C,Polymer_mass_N,Polymer_mass_O,Polymer_mass_F,Polymer_mass_P,Polymer_mass_S,Polymer_mass_Cl,Polymer_mass_Br,Polymer_mass_I,...,Solvent_fr_sulfide,Solvent_fr_sulfonamd,Solvent_fr_sulfone,Solvent_fr_term_acetylene,Solvent_fr_tetrazole,Solvent_fr_thiazole,Solvent_fr_thiocyan,Solvent_fr_thiophene,Solvent_fr_unbrch_alkane,Solvent_fr_urea
0,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,0,0,0,0,0,0,0,0,0,0
1,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,0,0,0,0,0,0,0,0,0,0
2,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,0,0,0,0,0,0,0,0,0,0
3,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,0,0,0,0,0,0,0,0,0,0
4,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,0,0,0,0,0,0,0,0,0,0


In [12]:
desc_container.Chi = pd.read_csv(f"{DIRPATH_LOAD}/desc_Chi.csv", index_col=0)
print(desc_container.Chi.shape)
desc_container.Chi.head()

(1190, 794)


,Polymer_mass_H,Polymer_mass_C,Polymer_mass_N,Polymer_mass_O,Polymer_mass_F,Polymer_mass_P,Polymer_mass_S,Polymer_mass_Cl,Polymer_mass_Br,Polymer_mass_I,...,Solvent_fr_sulfide,Solvent_fr_sulfonamd,Solvent_fr_sulfone,Solvent_fr_term_acetylene,Solvent_fr_tetrazole,Solvent_fr_thiazole,Solvent_fr_thiocyan,Solvent_fr_thiophene,Solvent_fr_unbrch_alkane,Solvent_fr_urea
0,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,0,0,0,0,0,0,0,0,0,0
1,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,0,0,0,0,0,0,0,0,0,0
2,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,0,0,0,0,0,0,0,0,0,0
3,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,0,0,0,0,0,0,0,0,0,0
4,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,0,0,0,0,0,0,0,0,0,0


### knowledge-based information

- p: polymer
- s: solvent


- rd: rdkit descriptor
- ff: radonpy force field parameters

In [13]:
with open(f"{DIRPATH_LOAD}/desc_names.json", mode="r") as f:
    dict_desc_names: dict[
        Literal["p_ff", "p_rd", "s_ff", "s_rd"], list[str]
    ] = json.load(f)

In [14]:
with open(f"{DIRPATH_LOAD}/test_cv_idx.pkl", mode="rb") as f:
    test_cv_idx: dict[Literal["train", "test"], list[np.ndarray[Any, int]]] = (
        pickle.load(f)
    )

## Split data

In [15]:
# define index container
@dataclass
class IndexContainer:
    train: np.ndarray[Any, int]
    test: np.ndarray[Any, int]

### Exp-Chi data splitting

In [16]:
# test_cv_idx["train"][IDX_CV_TEST], test_cv_idx["test"][IDX_CV_TEST]

### COSMO data splitting

exclude test ps_pair cases first

In [17]:
# test_cv_idx["train"][IDX_CV_TEST].shape, test_cv_idx["test"][IDX_CV_TEST].shape
index_containers = TargetContainer(
    Chi=IndexContainer(
        train=test_cv_idx["train"][IDX_CV_TEST],
        test=test_cv_idx["test"][IDX_CV_TEST],
    )
)

In [18]:
# XXX: same pair has diffrent chi parameters?
print(data_container.COSMO[COLNAME_SOLUBILITY_POLYMER_PAIR].duplicated().sum())
print(data_container.PI[COLNAME_SOLUBILITY_POLYMER_PAIR].duplicated().sum())

445
424


In [19]:
# HACK: もしCOSMO等のデータに重複がなければこのやり方が一番エレガント。
# データに重複があるので、サンプルコードよりデータが減りすぎてしまう。
pd.concat(
    (
        data_container.COSMO[COLNAME_SOLUBILITY_POLYMER_PAIR],
        data_container.Chi[COLNAME_SOLUBILITY_POLYMER_PAIR],
    ),
    axis=0,
).reset_index(drop=True).drop_duplicates(keep="first").loc[
    : data_container.COSMO.shape[0]
    - 1  # `loc` includes end
].shape

(761,)

In [20]:
# Chiのテストデータに含まれていないものの中から`test_ratio`分のテストデータを取り出す。
# それ以外はすべて学習データに加える
_flag_COSMO_in_Chi = data_container.COSMO[
    COLNAME_SOLUBILITY_POLYMER_PAIR
].isin(
    data_container.Chi.loc[
        test_cv_idx["test"][IDX_CV_TEST],
        COLNAME_SOLUBILITY_POLYMER_PAIR,
    ]
)
_flag_COSMO_test = _flag_COSMO_in_Chi.copy()
_flag_COSMO_test.loc[
    rng.choice(
        _flag_COSMO_in_Chi.index[~_flag_COSMO_in_Chi],
        size=math.ceil(np.sum(~_flag_COSMO_in_Chi, axis=None) * test_ratio),
        replace=False,
    )
] = True
index_containers.COSMO = IndexContainer(
    train=data_container.COSMO.index[~_flag_COSMO_test].to_numpy(),
    test=data_container.COSMO.index[_flag_COSMO_test].to_numpy(),
)

In [21]:
# Chiのテストデータに含まれていないものの中から`test_ratio`分のテストデータを取り出す。
# それ以外はすべて学習データに加える
_flag_PI_in_Chi = data_container.PI[COLNAME_SOLUBILITY_POLYMER_PAIR].isin(
    data_container.Chi.loc[
        test_cv_idx["test"][IDX_CV_TEST],
        COLNAME_SOLUBILITY_POLYMER_PAIR,
    ]
)
_flag_PI_test = _flag_PI_in_Chi.copy()
_flag_PI_test.loc[
    rng.choice(
        _flag_PI_in_Chi.index[~_flag_PI_in_Chi],
        size=math.ceil(np.sum(~_flag_PI_in_Chi, axis=None) * test_ratio),
        replace=False,
    )
] = True
index_containers.PI = IndexContainer(
    train=data_container.PI.index[~_flag_PI_test],
    test=data_container.PI.index[_flag_PI_test],
)

## Process data for training

### Scale descriptors

異なるデータセットであったとしても同じ特徴量であれば同じスケーラーで標準化する。

In [22]:
desc_names_rdkit = dict_desc_names["p_rd"] + dict_desc_names["s_rd"]
desc_rdkit_all = pd.concat(
    (
        desc_container.PI.loc[index_containers.PI.train, desc_names_rdkit],
        desc_container.COSMO.loc[
            index_containers.COSMO.train, desc_names_rdkit
        ],
        desc_container.Chi.loc[index_containers.Chi.train, desc_names_rdkit],
    ),
    axis=0,
)
desc_rdkit_all.head()

,Polymer_MaxEStateIndex,Polymer_MinEStateIndex,Polymer_MaxAbsEStateIndex,Polymer_MinAbsEStateIndex,Polymer_qed,Polymer_MolWt,Polymer_HeavyAtomMolWt,Polymer_ExactMolWt,Polymer_NumValenceElectrons,Polymer_NumRadicalElectrons,...,Solvent_fr_sulfide,Solvent_fr_sulfonamd,Solvent_fr_sulfone,Solvent_fr_term_acetylene,Solvent_fr_tetrazole,Solvent_fr_thiazole,Solvent_fr_thiocyan,Solvent_fr_thiophene,Solvent_fr_unbrch_alkane,Solvent_fr_urea
2,6.426039,0.737513,6.426039,0.737513,0.227374,885.37,834.97,880.079778,280,0,...,0,0,0,0,0,0,0,0,0,0
3,6.426039,0.737513,6.426039,0.737513,0.227374,885.37,834.97,880.079778,280,0,...,0,0,0,0,0,0,0,0,0,0
4,6.426039,0.737513,6.426039,0.737513,0.227374,885.37,834.97,880.079778,280,0,...,0,0,0,0,0,0,0,0,0,0
5,6.426039,0.737513,6.426039,0.737513,0.227374,885.37,834.97,880.079778,280,0,...,0,0,0,0,0,0,0,0,0,0
7,6.426039,0.737513,6.426039,0.737513,0.227374,885.37,834.97,880.079778,280,0,...,0,0,0,0,0,0,0,0,1,0


In [23]:
scaler_X_rdkit_all = make_pipeline(
    VarianceThreshold(), PowerTransformer(method="yeo-johnson")
).fit(desc_rdkit_all)
scaler_X_rdkit_all

Pipeline(steps=[('variancethreshold', VarianceThreshold()),
                ('powertransformer', PowerTransformer())])

Filter out constant descriptors (based on training data)

In [24]:
desc_names_forcefield = dict_desc_names["p_ff"] + dict_desc_names["s_ff"]
desc_forcefield_all = pd.concat(
    (
        desc_container.PI.loc[
            index_containers.PI.train, desc_names_forcefield
        ],
        desc_container.COSMO.loc[
            index_containers.COSMO.train, desc_names_forcefield
        ],
        desc_container.Chi.loc[
            index_containers.Chi.train, desc_names_forcefield
        ],
    ),
    axis=0,
)
desc_forcefield_all.head()

,Polymer_mass_H,Polymer_mass_C,Polymer_mass_N,Polymer_mass_O,Polymer_mass_F,Polymer_mass_P,Polymer_mass_S,Polymer_mass_Cl,Polymer_mass_Br,Polymer_mass_I,...,Solvent_k_dih_10,Solvent_k_dih_11,Solvent_k_dih_12,Solvent_k_dih_13,Solvent_k_dih_14,Solvent_k_dih_15,Solvent_k_dih_16,Solvent_k_dih_17,Solvent_k_dih_18,Solvent_k_dih_19
2,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,2.145700e-01,9.703123e-01,8.036671e-02,1.219166e-04,3.387440e-09,1.723862e-15,1.606776e-23,2.743029e-33,8.576845e-45,4.911868e-58
3,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,1.279998e-70,5.784715e-87,4.788249e-105,7.259280e-125,2.015731e-146,1.025164e-169,9.549409e-195,1.629227e-221,5.091067e-250,2.913789e-280
4,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
5,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,5.825000e-73,1.327453e-89,5.541533e-108,4.237384e-128,5.934810e-150,1.522468e-173,7.153478e-199,6.156180e-226,9.703515e-255,2.801366e-285
7,0.586817,0.508618,0.439979,0.357449,0.227742,0.081921,0.089032,0.10039,1.485486e-12,1.472831e-47,...,1.534702e-24,3.374939e-34,1.359345e-45,1.002803e-58,1.354951e-73,3.353156e-90,1.519868e-108,1.261770e-128,1.918565e-150,5.343119e-174


In [25]:
selector_X_ff_all = VarianceThreshold().fit(desc_forcefield_all)
selector_X_ff_all

VarianceThreshold()

Setup and scale y values and temperatures

s: source, t: target

- `y_s0`: `y_container.PI`
- `y_s`: `y_container.COSMO`
- `y_t`: `y_container.Chi`

In [26]:
y_container = TargetContainer(
    PI=data_container.PI[["soluble"]],
    COSMO=data_container.COSMO[["chi"]],
    Chi=data_container.Chi[["chi"]],
)

In [27]:
def convert_temperature(
    sr: pd.Series, temp_dim: Literal[1, 2]
) -> pd.DataFrame:
    return pd.concat(
        (
            (1 / ((sr + zero_Celsius) ** _dim)).rename(f"T{_dim}")
            for _dim in range(1, temp_dim + 1)
        ),
        axis=1,
    )


temperature_container = TargetContainer(
    COSMO=convert_temperature(data_container.COSMO["temp"], temp_dim=temp_dim),
    Chi=convert_temperature(data_container.Chi["temp"], temp_dim=temp_dim),
)
temperature_container.COSMO.head()

,T1
0,0.00268
1,0.00268
2,0.00268
3,0.00268
4,0.00268


In [28]:
scaler_y_container = TargetContainer(
    COSMO=StandardScaler(), Chi=StandardScaler()
)

y_scaled_container = TargetContainer(PI=y_container.PI)
for _key in scaler_y_container.keys():
    if isinstance(getattr(scaler_y_container, _key), StandardScaler):
        _scaler_y: StandardScaler = getattr(scaler_y_container, _key)
        _y: pd.DataFrame = getattr(y_container, _key)
        _index_container: IndexContainer = getattr(index_containers, _key)
        _scaler_y.fit(_y.loc[_index_container.train])
        setattr(
            y_scaled_container,
            _key,
            pd.DataFrame(
                _scaler_y.transform(_y), index=_y.index, columns=_y.columns
            ),
        )

In [29]:
temperature_scaled_container: TargetContainer[pd.DataFrame] = TargetContainer()

scaler_temperature_container = TargetContainer(
    COSMO=StandardScaler(), Chi=StandardScaler()
)
for _key in scaler_temperature_container.keys():
    _scaler_temperature: StandardScaler = getattr(
        scaler_temperature_container, _key
    )
    _df_temperature: pd.DataFrame = getattr(temperature_container, _key)
    _index_container: IndexContainer = getattr(index_containers, _key)

    _scaler_temperature.fit(_df_temperature.loc[_index_container.train])
    setattr(
        temperature_scaled_container,
        _key,
        pd.DataFrame(
            _scaler_temperature.transform(_df_temperature),
            index=_df_temperature.index,
            columns=_df_temperature.columns,
        ),
    )

## Build model

In [30]:
# fixed linearly reducing pyramid shape
def neuron_vector(nL: int, in_neu: int, out_neu: int) -> tuple[int]:
    return tuple(
        int(x) for x in np.rint(np.linspace(in_neu, out_neu, nL + 2))[1:-1]
    )

In [31]:
torch.cat(
    (torch.zeros((4, 1)), torch.tensor(np.arange(12).reshape(4, 3))), dim=1
)

tensor([[ 0.,  0.,  1.,  2.],
        [ 0.,  3.,  4.,  5.],
        [ 0.,  6.,  7.,  8.],
        [ 0.,  9., 10., 11.]])

In [32]:
class ChiModel(nn.Module):
    def __init__(
        self,
        model_polymer: nn.Module,
        model_solvent: nn.Module,
        dim_ur: int,  # おそらく、uとrの区切れがどこにあるかを表すパラメータ
        temp_dim: Literal[1, 2],
    ) -> None:
        super().__init__()

        self.model_polymer = deepcopy(model_polymer)
        self.model_solvent = deepcopy(model_solvent)
        self.dim_ur = dim_ur
        self.temp_dim = temp_dim

        self.fc = nn.Linear(self.dim_ur, 3 + 2 * self.temp_dim)
        """fully connected layer"""
        self.activation_function = nn.Sigmoid()

    def forward(
        self,
        x_polymer: torch.Tensor,
        x_solvent: torch.Tensor,
        x_temp: torch.Tensor,
    ) -> torch.Tensor:
        assert x_temp.shape[1] == self.temp_dim

        # feature embedding
        ur_polymer = self.model_polymer(x_polymer)
        ur_solvent = self.model_solvent(x_solvent)

        # > The first term calculates the elementwise squared difference of the k-dimensional latent vectors,
        # u_p and u_s.
        sp = torch.square(
            ur_polymer[:, : self.dim_ur] - ur_solvent[:, : self.dim_ur]
        )

        # > The second and third terms denote the polymer- and solvent-specific discount factors, respectively,
        # which are analogous to the interaction radius of the HSP sphere.
        r_polymer = torch.square(ur_polymer[:, self.dim_ur :])
        r_solvent = torch.square(ur_solvent[:, self.dim_ur :])

        # equation 4 in paper; distance operator
        z_raw = sp - r_polymer - r_solvent
        z = self.fc(z_raw)

        z_soluble = self.activation_function(z[:, [0]])

        # おそらく、温度を線型外挿 (temp_dimが2の場合は2次の項もいれて) しようとしている。
        z_cosmo = torch.sum(
            torch.cat((torch.ones((x_temp.shape[0], 1)), x_temp), dim=1)
            * z[1 : 2 + self.temp_dim],
            dim=1,
        )
        z_chi = torch.sum(
            torch.cat((torch.ones((x_temp.shape[0], 1)), x_temp), dim=1)
            * z[2 + self.temp_dim : 3 + 2 * self.temp_dim],
            dim=1,
        )

        return torch.cat((z_soluble, z_cosmo, z_chi, z, z_raw), dim=1)

In [33]:
class LinearLayer(nn.Module):
    """
    Base NN layer. This is a wrap around PyTorch.
    See here for details: http://pytorch.org/docs/master/nn.html#
    """

    def __init__(
        self,
        in_features: int,
        out_features: int,
        *,
        bias: bool = True,
        dropout: float = 0.0,
        activation_func: Callable = nn.ReLU(),
        normalizer: Optional[float] = 0.1,
    ):
        """
        Parameters
        ----------
        in_features:
            Size of each input sample.
        out_features:
            Size of each output sample
        bias:
            If set to ``False``, the layer will not learn an additive bias.
            Default: ``True``
        dropout: float
            Probability of an element to be zeroed. Default: 0.5
        activation_func: func
            Activation function.
        normalizer: func
            Normalization layers
        """
        super().__init__()
        self.linear = nn.Linear(in_features, out_features, bias)
        self.dropout = nn.Dropout(dropout)
        self.normalizer = (
            None
            if not normalizer
            else nn.BatchNorm1d(out_features, momentum=normalizer)
        )
        self.activation = None if not activation_func else activation_func

    def forward(self, x):
        _out = self.linear(x)
        if self.dropout:
            _out = self.dropout(_out)
        if self.normalizer:
            _out = self.normalizer(_out)
        if self.activation:
            _out = self.activation(_out)
        return _out


class SequentialLinear(nn.Module):
    """
    Sequential model with linear layers and configurable other hype-parameters.
    e.g. ``dropout``, ``hidden layers``

    """

    def __init__(
        self,
        in_features: int,
        out_features: int,
        bias: bool = True,
        *,
        h_neurons: Union[Sequence[float], Sequence[int]] = (),
        h_bias: Union[bool, Sequence[bool]] = True,
        h_dropouts: Union[float, Sequence[float]] = 0.0,
        h_normalizers: Union[float, None, Sequence[Optional[float]]] = 0.1,
        h_activation_funcs: Union[
            Callable, None, Sequence[Optional[Callable]]
        ] = nn.ReLU(),
    ):
        """

        Parameters
        ----------
        in_features
            Size of input.
        out_features
            Size of output.
        bias
            Enable ``bias`` in input layer.
        h_neurons
            Number of neurons in hidden layers.
            Can be a tuple of floats. In that case,
            all these numbers will be used to calculate the neuron numbers.
            e.g. (0.5, 0.4, ...) will be expanded as (in_features * 0.5, in_features * 0.4, ...)
        h_bias
            ``bias`` in hidden layers.
        h_dropouts
            Probabilities of dropout in hidden layers.
        h_normalizers
            Momentum of batched normalizers in hidden layers.
        h_activation_funcs
            Activation functions in hidden layers.
        """
        super().__init__()
        self._h_layers = len(h_neurons)
        if self._h_layers > 0:
            if isinstance(h_neurons[0], float):
                tmp = [in_features]
                for i, ratio in enumerate(h_neurons):
                    num = math.ceil(in_features * ratio)
                    tmp.append(num)
                neurons = tuple(tmp)

            elif isinstance(h_neurons[0], int):
                neurons = (in_features,) + tuple(h_neurons)
            else:
                raise RuntimeError("illegal parameter type of <h_neurons>")

            activation_funcs = self._check_input(h_activation_funcs)
            normalizers = self._check_input(h_normalizers)
            dropouts = self._check_input(h_dropouts)
            bias = (bias,) + self._check_input(h_bias)

            for i in range(self._h_layers):
                setattr(
                    self,
                    f"layer_{i}",
                    LinearLayer(
                        in_features=neurons[i],
                        out_features=neurons[i + 1],
                        bias=bias[i],
                        dropout=dropouts[i],
                        activation_func=activation_funcs[i],
                        normalizer=normalizers[i],
                    ),
                )

            self.output = nn.Linear(neurons[-1], out_features, bias[-1])
        else:
            self.output = nn.Linear(in_features, out_features, bias)

    def _check_input(self, i):
        if isinstance(i, Sequence):
            if len(i) != self._h_layers:
                raise RuntimeError(
                    f"number of parameter not consistent with number of layers, "
                    f"input is {len(i)} but need to be {self._h_layers}"
                )
            return tuple(i)
        else:
            return tuple([i] * self._h_layers)

    def forward(self, x: Any) -> Any:
        for i in range(self._h_layers):
            x = getattr(self, f"layer_{i}")(x)
        return self.output(x)

In [34]:
# functions to save and load the NN model
def save_NN(
    paras_p: dict,
    paras_s: dict,
    dim_out: int,
    c_mdl: nn.Module,
    file_name: str,
):
    torch.save(
        {
            "model_p": paras_p,
            "model_s": paras_s,
            "chi": c_mdl.state_dict(),
            "dim_out": dim_out,
        },
        file_name,
    )


def load_NN(file_name: str):
    tmp_paras = torch.load(file_name)
    c_model = ChiModel(
        SequentialLinear(**tmp_paras["model_p"]),
        SequentialLinear(**tmp_paras["model_s"]),
        tmp_paras["dim_out"],
    )
    _ = c_model.load_state_dict(tmp_paras["chi"])
    return c_model

## Hyperparameter tuning with grid search and CV

まずはCOSMOとPIのデータで学習する

Group validation

In [35]:
groups_polymer: list[str] = data_container.Chi.loc[
    index_containers.Chi.train, COLNAME_SOLUBILITY_POLYMER_PAIR
].tolist()

In [36]:
group_kf = GroupKFold(n_splits=n_CV_val)

index_containers_hyperparams_tuning: list[IndexContainer] = list()
_y_chi_train: pd.DataFrame = y_container.Chi.loc[index_containers.Chi.train]
for _index_train, _index_val in group_kf.split(
    _y_chi_train, groups=groups_polymer
):
    index_containers_hyperparams_tuning.append(
        IndexContainer(
            train=_y_chi_train.iloc[_index_train].index.to_numpy(),
            test=_y_chi_train.iloc[_index_val].index.to_numpy(),
        )
    )

Prepare for training

In [37]:
# NOTE: 特徴量の取り出し方が違うので注意。
# desc_names_polymer: tuple[str] = tuple(
#     _desc_name
#     for _desc_name in desc_names_forcefield
#     if "Polymer" in _desc_name
# ) + tuple(
#     _desc_name for _desc_name in desc_names_rdkit if "Polymer" in _desc_name
# )
# desc_names_solvent: tuple[str] = tuple(
#     _desc_name
#     for _desc_name in desc_names_forcefield
#     if "Solvent" in _desc_name
# ) + tuple(
#     _desc_name for _desc_name in desc_names_rdkit if "Solvent" in _desc_name
# )
# desc_names_polymer[:5], desc_names_solvent[:5]

In [38]:
assert desc_container.PI.shape[1] == len(desc_names_forcefield) + len(
    desc_names_rdkit
)
desc_names_use = (
    np.hstack(
        (
            selector_X_ff_all.get_feature_names_out(),
            scaler_X_rdkit_all.get_feature_names_out(),
        )
    )
    .astype(str)
    .tolist()
)

In [76]:
@dataclass
class DatasetLikeObject:
    x_polymer: torch.Tensor
    x_solvent: torch.Tensor
    x_temp: torch.Tensor
    y: torch.Tensor


@dataclass
class DatasetContainer:
    train: DatasetLikeObject
    test: DatasetLikeObject

In [75]:
datasets: TargetContainer[DatasetContainer] = TargetContainer()
for _key in ("PI", "COSMO", "Chi"):
    _desc: pd.DataFrame = getattr(desc_container, _key)
    for _train_or_test in ("train", "test"):
        _index = getattr(index_containers[_key], _train_or_test)
        _X = np.concatenate(
            (
                selector_X_ff_all.transform(
                    _desc.loc[_index, desc_names_forcefield]
                ),
                scaler_X_rdkit_all.transform(
                    _desc.loc[_index, desc_names_rdkit]
                ),
            ),
            axis=1,
        ).astype(np.float32)

        assert _X.shape[1] == len(desc_names_use)

        _X_polymer = torch.tensor(
            _X[
                :,
                np.char.startswith(
                    desc_names_use,
                    "Polymer_",
                ),
            ],
            device=device,
        )
        _X_solvent = torch.tensor(
            _X[
                :,
                np.char.startswith(
                    desc_names_use,
                    "Solvent_",
                ),
            ],
            device=device,
        )

        _y_scaled = torch.tensor(
            y_scaled_container[_key].loc[_index].values,
            dtype=torch.float32,
            device=device,
        )

        setattr(
            getattr(datasets, _key),
            _train_or_test,
            DatasetContainer(
                train=DatasetLikeObject(
                    x_polymer=_X_polymer, x_solvent=_X_solvent, x_temp=, y=_y_scaled,
                )
            ),
        )

tensor([[ 0.0434,  0.2008,  0.1926,  ..., -0.0568, -0.0753, -0.4971],
        [ 0.2434,  0.2440,  0.2157,  ..., -0.0568, -0.0753, -0.4971],
        [ 0.2390,  0.3700,  0.3383,  ..., -0.0568, -0.0753, -0.4971],
        ...,
        [ 0.6441,  0.5729,  0.4969,  ..., -0.0568, -0.0753, -0.4971],
        [ 0.5640,  0.6397,  0.5771,  ..., -0.0568, 13.2853, -0.4971],
        [ 0.5640,  0.6397,  0.5771,  ..., -0.0568, 13.2853, -0.4971]])
tensor([[ 0.0434,  0.2008,  0.1926,  ..., -0.0568, -0.0753, -0.4971],
        [ 0.2434,  0.2440,  0.2157,  ..., -0.0568, -0.0753, -0.4971],
        [ 0.2390,  0.3700,  0.3383,  ..., -0.0568, -0.0753, -0.4971],
        ...,
        [ 0.6046,  0.3913,  0.3246,  ..., -0.0568, -0.0753, -0.4971],
        [ 0.7302,  0.4679,  0.3961,  ..., -0.0568, -0.0753, -0.4971],
        [ 0.6710,  0.5120,  0.4516,  ..., -0.0568, -0.0753, -0.4971]])
tensor([[ 0.0434,  0.2008,  0.1926,  ..., -0.0568, -0.0753, -0.4971],
        [ 0.2434,  0.2440,  0.2157,  ..., -0.0568, -0.0753, -0

In [ ]:
criterion_container = TargetContainer(
    PI=nn.BCELoss(), COSMO=nn.MSELoss(), Chi=nn.MSELoss()
)
# dim_in_p = len(dname_p)
# dim_in_s = len(dname_s)


# HACK: replace random sampling to optuna
# generate hyperparameters candidates
hyper_para = pd.DataFrame(
    {
        "alpha1": rng.uniform(alpha1s[0], alpha1s[1], n_hpara),
        "alpha2": rng.uniform(alpha2s[0], alpha2s[1], n_hpara),
        "dim_out": rng.integers(dim_outs[0], dim_outs[1], n_hpara),
        "lr": rng.uniform(learning_rates[0], learning_rates[1], n_hpara),
    }
)

os.makedirs(dir_base, exist_ok=True)
hyper_para.to_csv(f"{dir_base}/list_hyperparameters.csv")


XS0_P_TE = torch.tensor(
    desc_s0_s.loc[idx_split_s0["idx_te"], dname_p].values.astype("float"),
    dtype=torch.float32,
    device=device,
)
XS0_S_TE = torch.tensor(
    desc_s0_s.loc[idx_split_s0["idx_te"], dname_s].values.astype("float"),
    dtype=torch.float32,
    device=device,
)
XS0_T_TE = torch.zeros(
    idx_split_s0["idx_te"].shape[0], temp_dim, device=device
)
YS0_TE = torch.tensor(
    y_s0.loc[idx_split_s0["idx_te"], :].values.astype("float"),
    dtype=torch.float32,
    device=device,
)

XS_P_TE = torch.tensor(
    desc_s_s.loc[idx_split_s["idx_te"], dname_p].values.astype("float"),
    dtype=torch.float32,
    device=device,
)
XS_S_TE = torch.tensor(
    desc_s_s.loc[idx_split_s["idx_te"], dname_s].values.astype("float"),
    dtype=torch.float32,
    device=device,
)
XS_T_TE = torch.tensor(
    temp_s_s.loc[idx_split_s["idx_te"], :].values.astype("float"),
    dtype=torch.float32,
    device=device,
)
YS_TE = torch.tensor(
    y_s_s.loc[idx_split_s["idx_te"], :].values.astype("float"),
    dtype=torch.float32,
    device=device,
)

XT_P_TE = torch.tensor(
    desc_t_s.loc[idx_split_t["idx_te"], dname_p].values.astype("float"),
    dtype=torch.float32,
    device=device,
)
XT_S_TE = torch.tensor(
    desc_t_s.loc[idx_split_t["idx_te"], dname_s].values.astype("float"),
    dtype=torch.float32,
    device=device,
)
XT_T_TE = torch.tensor(
    temp_t_s.loc[idx_split_t["idx_te"], :].values.astype("float"),
    dtype=torch.float32,
    device=device,
)
YT_TE = torch.tensor(
    y_t_s.loc[idx_split_t["idx_te"], :].values.astype("float"),
    dtype=torch.float32,
    device=device,
)

XS0_P_TR = torch.tensor(
    desc_s0_s.loc[idx_split_s0["idx_tr"], dname_p].values.astype("float"),
    dtype=torch.float32,
    device=device,
)
XS0_S_TR = torch.tensor(
    desc_s0_s.loc[idx_split_s0["idx_tr"], dname_s].values.astype("float"),
    dtype=torch.float32,
    device=device,
)
XS0_T_TR = torch.zeros(
    idx_split_s0["idx_tr"].shape[0], temp_dim, device=device
)
YS0_TR = torch.tensor(
    y_s0.loc[idx_split_s0["idx_tr"], :].values.astype("float"),
    dtype=torch.float32,
    device=device,
)

XS_P_TR = torch.tensor(
    desc_s_s.loc[idx_split_s["idx_tr"], dname_p].values.astype("float"),
    dtype=torch.float32,
    device=device,
)
XS_S_TR = torch.tensor(
    desc_s_s.loc[idx_split_s["idx_tr"], dname_s].values.astype("float"),
    dtype=torch.float32,
    device=device,
)
XS_T_TR = torch.tensor(
    temp_s_s.loc[idx_split_s["idx_tr"], :].values.astype("float"),
    dtype=torch.float32,
    device=device,
)
YS_TR = torch.tensor(
    y_s_s.loc[idx_split_s["idx_tr"], :].values.astype("float"),
    dtype=torch.float32,
    device=device,
)

Training models

In [ ]:
class ParameterGenerator(object):
    """
    Generator for parameter set generating.

    """

    def __init__(
        self,
        seed: Optional[int] = None,
        **kwargs: Union[Any, Sequence, Callable, dict],
    ):
        """

        Parameters
        ----------
        seed
            Numpy random seed.
        kwargs
            Parameter candidate.
        """
        if len(kwargs) == 0:
            raise RuntimeError("need parameter candidate")

        self.rng = np.random.default_rng(seed)

        self.tuples = dict()
        self.funcs = dict()
        self.dicts = dict()
        self.others = dict()

        for k, v in kwargs.items():
            if isinstance(v, (tuple, list, np.ndarray, pd.Series)):
                self.tuples[k] = v
            elif callable(v):
                self.funcs[k] = v
            elif isinstance(v, dict):
                repeat = v["repeat"]
                self.dicts[k] = v

                if isinstance(repeat, str):
                    if repeat in self.tuples:
                        self.tuples[repeat] = self.tuples.pop(repeat)
                    if repeat in self.dicts:
                        self.dicts[repeat] = self.dicts.pop(repeat)
                    if repeat in self.funcs:
                        self.funcs[repeat] = self.funcs.pop(repeat)
            else:
                self.others[k] = v

    def __call__(self, num: int, *, factory=None):
        for _ in range(num):
            tmp = {}
            for k, v in self.tuples.items():
                tmp[k] = self._gen(v)

            for k, v in self.funcs.items():
                tmp[k] = v()

            for k, v in reversed(self.dicts.items()):
                data = v["data"]
                repeat = v["repeat"]
                if "replace" in v:
                    replace = v["replace"]
                else:
                    replace = True

                if isinstance(repeat, (tuple, list, np.ndarray, pd.Series)):
                    repeat = self._gen(repeat)
                elif isinstance(repeat, str):
                    repeat = len(tmp[repeat])

                if isinstance(data, (tuple, list, np.ndarray, pd.Series)):
                    tmp[k] = self._gen(data, repeat, replace)
                elif callable(data):
                    tmp[k] = tuple(data(repeat))

            tmp = dict(self.others, **tmp)
            if factory is not None:
                yield tmp, factory(**tmp)
            else:
                yield tmp

    @staticmethod
    def _gen(item: Sequence, repeat: int = None, replace: bool = True):
        if repeat is not None:
            idx = rng.choice(len(item), repeat, replace=replace)
            return tuple(item[i] for i in idx)
        else:
            idx = rng.choice(len(item))
            return item[idx]

In [ ]:
for iCV, (idx_tr, idx_val) in enumerate(zip(idx_trs, idx_vals)):
    XT_P_TR = torch.tensor(
        desc_t_s.loc[idx_tr, dname_p].values.astype("float"),
        dtype=torch.float32,
        device=device,
    )
    XT_S_TR = torch.tensor(
        desc_t_s.loc[idx_tr, dname_s].values.astype("float"),
        dtype=torch.float32,
        device=device,
    )
    XT_T_TR = torch.tensor(
        temp_t_s.loc[idx_tr, :].values.astype("float"),
        dtype=torch.float32,
        device=device,
    )
    YT_TR = torch.tensor(
        y_t_s.loc[idx_tr, :].values.astype("float"),
        dtype=torch.float32,
        device=device,
    )

    XT_P_VA = torch.tensor(
        desc_t_s.loc[idx_val, dname_p].values.astype("float"),
        dtype=torch.float32,
        device=device,
    )
    XT_S_VA = torch.tensor(
        desc_t_s.loc[idx_val, dname_s].values.astype("float"),
        dtype=torch.float32,
        device=device,
    )
    XT_T_VA = torch.tensor(
        temp_t_s.loc[idx_val, :].values.astype("float"),
        dtype=torch.float32,
        device=device,
    )
    YT_VA = torch.tensor(
        y_t_s.loc[idx_val, :].values.astype("float"),
        dtype=torch.float32,
        device=device,
    )

    for ii, h_paras in hyper_para.iterrows():
        alpha1 = h_paras["alpha1"]
        alpha2 = h_paras["alpha2"]
        dim_out = int(h_paras["dim_out"])
        learning_rate = h_paras["lr"]

        generator_p = ParameterGenerator(
            in_features=dim_in_p,
            out_features=dim_out * 2,
            h_neurons=dict(
                data=lambda x: neuron_vector(x, dim_in_p, dim_out * 2),
                repeat=[n_NNlayer],
            ),
            h_activation_funcs=(nn.Sigmoid(),),
            h_dropouts=(0.0,),
        )

        generator_s = ParameterGenerator(
            in_features=dim_in_s,
            out_features=dim_out * 2,
            h_neurons=dict(
                data=lambda x: neuron_vector(x, dim_in_s, dim_out * 2),
                repeat=[n_NNlayer],
            ),
            h_activation_funcs=(nn.Sigmoid(),),
            h_dropouts=(0.0,),
        )

        for iM, ((paras_p, model_p), (paras_s, model_s)) in enumerate(
            zip(
                generator_p(num=1, factory=SequentialLinear),
                generator_s(num=1, factory=SequentialLinear),
            )
        ):
            dir_save = f"{dir_base}/cv{iCV}/hpara{ii}"
            os.makedirs(dir_save, exist_ok=True)

            c_model = ChiModel(model_p, model_s, dim_out)
            _ = c_model.to(device)

            _ = c_model.train()

            optimizer = optim.Adam(
                c_model.parameters(), lr=learning_rate, amsgrad=True
            )
            scheduler = StepLR(
                optimizer, step_size=sch_step_size, gamma=sch_gamma
            )

            learning_curve = pd.DataFrame()

            for t in range(epochs_s):
                # mini-batch of training data
                kf = KFold(
                    n_splits=n_minibatch_PI,
                    shuffle=True,
                    random_state=rng.integers(2**31 - 1),
                )
                idx_mb_s0 = [x for _, x in kf.split(XS0_P_TR)]

                # pre-training with PI
                for tt, ii_s0 in enumerate(idx_mb_s0):
                    _ = c_model.train()

                    tmp_source0_train = c_model(
                        XS0_P_TR[ii_s0, :],
                        XS0_S_TR[ii_s0, :],
                        XS0_T_TR[ii_s0, :],
                    )
                    py_source0_train = tmp_source0_train[:, 0:1]
                    loss_source0_train = criterion_source0(
                        py_source0_train, YS0_TR[ii_s0, :]
                    )

                    optimizer.zero_grad()
                    loss_source0_train.backward()
                    optimizer.step()

                    _ = c_model.eval()
                    with torch.no_grad():
                        py_source_train = c_model(XS_P_TR, XS_S_TR, XS_T_TR)[
                            :, 1:2
                        ]
                        py_target_train = c_model(XT_P_TR, XT_S_TR, XT_T_TR)[
                            :, 2:3
                        ]
                        loss_source_train = criterion_source(
                            py_source_train, YS_TR
                        )
                        loss_target_train = (
                            criterion_target(py_target_train, YT_TR)
                            * loss_factor_target
                        )
                        loss_train = alpha1 * loss_source0_train + (
                            1.0 - alpha1
                        ) * (
                            alpha2 * loss_source_train
                            + (1.0 - alpha2) * loss_target_train
                        )

                        py_target_val = c_model(XT_P_VA, XT_S_VA, XT_T_VA)[
                            :, 1:2
                        ]
                        loss_target_val = (
                            criterion_target(py_target_val, YT_VA)
                            * loss_factor_target
                        )

                        py_source0_test = c_model(
                            XS0_P_TE, XS0_S_TE, XS0_T_TE
                        )[:, 0:1]
                        py_source_test = c_model(XS_P_TE, XS_S_TE, XS_T_TE)[
                            :, 1:2
                        ]
                        py_target_test = c_model(XT_P_TE, XT_S_TE, XT_T_TE)[
                            :, 2:3
                        ]
                        loss_source0_test = criterion_source0(
                            py_source0_test, YS0_TE
                        )
                        loss_source_test = criterion_source(
                            py_source_test, YS_TE
                        )
                        loss_target_test = (
                            criterion_target(py_target_test, YT_TE)
                            * loss_factor_target
                        )
                        loss_test = alpha1 * loss_source0_test + (
                            1.0 - alpha1
                        ) * (
                            alpha2 * loss_source_test
                            + (1.0 - alpha2) * loss_target_test
                        )

                    learning_curve = pd.concat(
                        [
                            learning_curve,
                            pd.Series(
                                {
                                    "Loss_Source0_Training": loss_source0_train.item(),
                                    "Loss_Source0_Test": loss_source0_test.item(),
                                    "Loss_Source_Training": loss_source_train.item(),
                                    "Loss_Source_Test": loss_source_test.item(),
                                    "Loss_Target_Training": loss_target_train.item(),
                                    "Loss_Target_Validation": loss_target_val.item(),
                                    "Loss_Target_Test": loss_target_test.item(),
                                    "Loss_Training": loss_train.item(),
                                    "Loss_Test": loss_test.item(),
                                },
                                name=f"pre_{t}",
                            )
                            .to_frame()
                            .T,
                        ],
                        axis=0,
                    )

            # main training
            best_loss_val = np.inf
            for t in range(epochs):
                # mini-batch of training data
                kf = KFold(
                    n_splits=n_minibatch_PI,
                    shuffle=True,
                    random_state=rng.integers(2**31 - 1),
                )
                idx_mb_s0 = [x for _, x in kf.split(XS0_P_TR)]
                idx_mb_s, idx_mb_t = [], []
                for k in range(n_factor_COSMO):
                    kf = KFold(
                        n_splits=n_minibatch_COSMO,
                        shuffle=True,
                        random_state=rng.integers(2**31 - 1),
                    )
                    idx_mb_s += [x for _, x in kf.split(XS_P_TR)]
                for k in range(n_factor_CHI):
                    kf = KFold(
                        n_splits=n_minibatch_CHI,
                        shuffle=True,
                        random_state=rng.integers(2**31 - 1),
                    )
                    idx_mb_t += [x for _, x in kf.split(XT_P_TR)]

                for tt, (ii_s0, ii_s, ii_t) in enumerate(
                    zip(idx_mb_s0, idx_mb_s, idx_mb_t)
                ):
                    _ = c_model.train()
                    if alpha1 > 0:
                        py_source0_train = c_model(
                            XS0_P_TR[ii_s0, :],
                            XS0_S_TR[ii_s0, :],
                            XS0_T_TR[ii_s0, :],
                        )[:, 0:1]
                        loss_source0_train = criterion_source0(
                            py_source0_train, YS0_TR[ii_s0, :]
                        )
                    else:
                        loss_source0_train = torch.zeros(1, device=device)

                    if (alpha2 > 0) and (alpha1 < 1):
                        if no_COSMO_BN:
                            _ = c_model.eval()
                        else:
                            _ = c_model.train()
                        py_source_train = c_model(
                            XS_P_TR[ii_s, :],
                            XS_S_TR[ii_s, :],
                            XS_T_TR[ii_s, :],
                        )[:, 1:2]
                        loss_source_train = criterion_source(
                            py_source_train, YS_TR[ii_s, :]
                        )
                    else:
                        loss_source_train = torch.zeros(1, device=device)

                    if (alpha1 < 1) and (alpha2 < 1):
                        if no_target_BN:
                            _ = c_model.eval()
                        else:
                            _ = c_model.train()
                        py_target_train = c_model(
                            XT_P_TR[ii_t, :],
                            XT_S_TR[ii_t, :],
                            XT_T_TR[ii_t, :],
                        )[:, 2:3]
                        loss_target_train = (
                            criterion_target(py_target_train, YT_TR[ii_t, :])
                            * loss_factor_target
                        )
                    else:
                        loss_target_train = torch.zeros(1, device=device)

                    loss_train = alpha1 * loss_source0_train + (
                        1.0 - alpha1
                    ) * (
                        alpha2 * loss_source_train
                        + (1.0 - alpha2) * loss_target_train
                    )

                    optimizer.zero_grad()
                    loss_train.backward()
                    optimizer.step()

                    _ = c_model.eval()
                    with torch.no_grad():
                        py_target_val = c_model(XT_P_VA, XT_S_VA, XT_T_VA)[
                            :, 2:3
                        ]
                        loss_target_val = (
                            criterion_target(py_target_val, YT_VA)
                            * loss_factor_target
                        )

                        py_source0_test = c_model(
                            XS0_P_TE, XS0_S_TE, XS0_T_TE
                        )[:, 0:1]
                        py_source_test = c_model(XS_P_TE, XS_S_TE, XS_T_TE)[
                            :, 1:2
                        ]
                        py_target_test = c_model(XT_P_TE, XT_S_TE, XT_T_TE)[
                            :, 2:3
                        ]
                        loss_source0_test = criterion_source0(
                            py_source0_test, YS0_TE
                        )
                        loss_source_test = criterion_source(
                            py_source_test, YS_TE
                        )
                        loss_target_test = (
                            criterion_target(py_target_test, YT_TE)
                            * loss_factor_target
                        )
                        loss_test = alpha1 * loss_source0_test + (
                            1.0 - alpha1
                        ) * (
                            alpha2 * loss_source_test
                            + (1.0 - alpha2) * loss_target_test
                        )

                    learning_curve = pd.concat(
                        [
                            learning_curve,
                            pd.Series(
                                {
                                    "Loss_Source0_Training": loss_source0_train.item(),
                                    "Loss_Source0_Test": loss_source0_test.item(),
                                    "Loss_Source_Training": loss_source_train.item(),
                                    "Loss_Source_Test": loss_source_test.item(),
                                    "Loss_Target_Training": loss_target_train.item(),
                                    "Loss_Target_Validation": loss_target_val.item(),
                                    "Loss_Target_Test": loss_target_test.item(),
                                    "Loss_Training": loss_train.item(),
                                    "Loss_Test": loss_test.item(),
                                },
                                name=f"main_{t}_{tt}",
                            )
                            .to_frame()
                            .T,
                        ],
                        axis=0,
                    )

                    if (t > burn_in) and (loss_target_val < best_loss_val):
                        save_NN(
                            paras_p,
                            paras_s,
                            dim_out,
                            c_model,
                            f"{dir_save}/best_loss_target_val.pt",
                        )

                        with torch.no_grad():
                            py_source0_train = c_model(
                                XS0_P_TR, XS0_S_TR, XS0_T_TR
                            )[:, 0:1]
                            py_source_train = c_model(
                                XS_P_TR, XS_S_TR, XS_T_TR
                            )[:, 1:2]
                            py_target_train = c_model(
                                XT_P_TR, XT_S_TR, XT_T_TR
                            )[:, 2:3]
                        pd.concat(
                            [
                                y_s0.loc[idx_split_s0["idx_tr"], :],
                                pd.Series(
                                    py_source0_train.to("cpu")
                                    .detach()
                                    .numpy()
                                    .flatten(),
                                    index=idx_split_s0["idx_tr"],
                                    name="pred",
                                ),
                            ],
                            axis=1,
                        ).to_csv(
                            f"{dir_save}/best_loss_target_val_source0_train.csv"
                        )
                        pd.concat(
                            [
                                y_s.loc[idx_split_s["idx_tr"], :],
                                pd.Series(
                                    ys_scaler.inverse_transform(
                                        py_source_train.to("cpu")
                                        .detach()
                                        .numpy()
                                    ).flatten(),
                                    index=idx_split_s["idx_tr"],
                                    name="pred",
                                ),
                            ],
                            axis=1,
                        ).to_csv(
                            f"{dir_save}/best_loss_target_val_source_train.csv"
                        )
                        pd.concat(
                            [
                                y_t.loc[idx_tr, :],
                                pd.Series(
                                    yt_scaler.inverse_transform(
                                        py_target_train.to("cpu")
                                        .detach()
                                        .numpy()
                                    ).flatten(),
                                    index=idx_tr,
                                    name="pred",
                                ),
                            ],
                            axis=1,
                        ).to_csv(
                            f"{dir_save}/best_loss_target_val_target_train.csv"
                        )

                        pd.concat(
                            [
                                y_t.loc[idx_val, :],
                                pd.Series(
                                    yt_scaler.inverse_transform(
                                        py_target_val.to("cpu")
                                        .detach()
                                        .numpy()
                                    ).flatten(),
                                    index=idx_val,
                                    name="pred",
                                ),
                            ],
                            axis=1,
                        ).to_csv(
                            f"{dir_save}/best_loss_target_val_target_val.csv"
                        )

                        pd.concat(
                            [
                                y_s0.loc[idx_split_s0["idx_te"], :],
                                pd.Series(
                                    py_source0_test.to("cpu")
                                    .detach()
                                    .numpy()
                                    .flatten(),
                                    index=idx_split_s0["idx_te"],
                                    name="pred",
                                ),
                            ],
                            axis=1,
                        ).to_csv(
                            f"{dir_save}/best_loss_target_val_source0_test.csv"
                        )
                        pd.concat(
                            [
                                y_s.loc[idx_split_s["idx_te"], :],
                                pd.Series(
                                    ys_scaler.inverse_transform(
                                        py_source_test.to("cpu")
                                        .detach()
                                        .numpy()
                                    ).flatten(),
                                    index=idx_split_s["idx_te"],
                                    name="pred",
                                ),
                            ],
                            axis=1,
                        ).to_csv(
                            f"{dir_save}/best_loss_target_val_source_test.csv"
                        )
                        pd.concat(
                            [
                                y_t.loc[idx_split_t["idx_te"], :],
                                pd.Series(
                                    yt_scaler.inverse_transform(
                                        py_target_test.to("cpu")
                                        .detach()
                                        .numpy()
                                    ).flatten(),
                                    index=idx_split_t["idx_te"],
                                    name="pred",
                                ),
                            ],
                            axis=1,
                        ).to_csv(
                            f"{dir_save}/best_loss_target_val_target_test.csv"
                        )

                        best_loss_val = loss_target_val

                scheduler.step()

            learning_curve.to_csv(f"{dir_save}/learning_curve.csv")

    print(f"Finished model {iCV}")

## Extract CV results

In [ ]:
def regression_metrics(
    y_true: Union[np.ndarray, pd.Series], y_pred: Union[np.ndarray, pd.Series]
) -> dict:
    """
    Calculate most common regression scores.
    See Also: https://scikit-learn.org/stable/modules/model_evaluation.html

    Parameters
    ----------
    y_true
        True results.
    y_pred
        Predicted results.

    Returns
    -------
    OrderedDict
        An :class:`collections.OrderedDict` contains regression scores.
        These scores will be calculated: ``mae``, ``mse``, ``rmse``, ``r2``,
        ``pearsonr``, ``spearmanr``, ``p_value``, and ``max_ae``
    """
    if len(y_true.shape) != 1:
        y_true = y_true.flatten()
    if len(y_pred.shape) != 1:
        y_pred = y_pred.flatten()

    mask = ~np.isnan(y_pred)
    y_true = y_true[mask]
    y_pred = y_pred[mask]

    mae = mean_absolute_error(y_true, y_pred)
    maxae = max_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    pr, p_val = pearsonr(y_true, y_pred)
    sr, _ = spearmanr(y_true, y_pred)
    return dict(
        mae=mae,
        mse=mse,
        rmse=rmse,
        r2=r2,
        pearsonr=pr,
        spearmanr=sr,
        p_value=p_val,
        max_ae=maxae,
    )


def classification_metrics(
    y_true: Union[np.ndarray, pd.DataFrame, pd.Series],
    y_pred: Union[np.ndarray, pd.Series],
    *,
    average: Optional[Sequence[Literal["weighted", "micro", "macro"]]] = (
        "weighted",
        "micro",
        "macro",
    ),
    labels=None,
) -> dict:
    """
    Calculate most common classification scores.
    See also: https://scikit-learn.org/stable/modules/model_evaluation.html

    Parameters
    ----------
    y_true
        True results.
    y_pred
        Predicted results.
    average
        This parameter is required for multiclass/multilabel targets. If None, the scores for each class are returned.
        Otherwise, this determines the type of averaging performed on the data:

        binary:
            Only report results for the class specified by pos_label. This is applicable only if targets (y_{true,pred})
            are binary.
        micro:
            Calculate metrics globally by counting the total true positives, false negatives and false positives.
        macro:
            Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into
            account.
        weighted:
            Calculate metrics for each label, and find their average weighted by support (the number of true instances
            for each label). This alters ``macro`` to account for label imbalance; it can result in an F-score that is
            not between precision and recall.
    labels
        The set of labels to include when average != ``binary``, and their order if average is None.
        Labels present in the data can be excluded, for example to calculate a multiclass average ignoring a majority
        negative class, while labels not present in the data will result in 0 components in a macro average.
        For multilabel targets, labels are column indices.
        By default, all labels in y_true and y_pred are used in sorted order.

    Returns
    -------
    OrderedDict
        An :class:`collections.OrderedDict` contains classification scores.
        These scores will always contains ``accuracy``, ``f1``, ``precision`` and ``recall``.
        For multilabel targets, based on the selection of the ``average`` parameter, the **weighted**, **micro**,
        and **macro** scores of ``f1`, ``precision``, and ``recall`` will be calculated.
    """
    if average is not None and len(average) == 0:
        raise ValueError("need average")

    if len(y_true.shape) != 1:
        y_true = np.argmax(y_true, 1)
    if len(y_pred.shape) != 1:
        y_pred = np.argmax(y_pred, 1)

    mask = ~np.isnan(y_pred)
    y_true = y_true[mask]
    y_pred = y_pred[mask]

    ret = dict(accuracy=accuracy_score(y_true, y_pred))

    ret.update(
        f1=f1_score(y_true, y_pred, average=None, labels=labels),
        precision=precision_score(y_true, y_pred, average=None, labels=labels),
        recall=recall_score(y_true, y_pred, average=None, labels=labels),
    )

    if "binary" in average:
        ret.update(
            binary_f1=f1_score(
                y_true, y_pred, average="binary", labels=labels
            ),
            binary_precision=precision_score(
                y_true, y_pred, average="binary", labels=labels
            ),
            binary_recall=recall_score(
                y_true, y_pred, average="binary", labels=labels
            ),
        )

    if "micro" in average:
        ret.update(
            micro_f1=f1_score(y_true, y_pred, average="micro", labels=labels),
            micro_precision=precision_score(
                y_true, y_pred, average="micro", labels=labels
            ),
            micro_recall=recall_score(
                y_true, y_pred, average="micro", labels=labels
            ),
        )

    if "macro" in average:
        ret.update(
            macro_f1=f1_score(y_true, y_pred, average="macro", labels=labels),
            macro_precision=precision_score(
                y_true, y_pred, average="macro", labels=labels
            ),
            macro_recall=recall_score(
                y_true, y_pred, average="macro", labels=labels
            ),
        )

    if "weighted" in average:
        ret.update(
            weighted_f1=f1_score(
                y_true, y_pred, average="weighted", labels=labels
            ),
            weighted_precision=precision_score(
                y_true, y_pred, average="weighted", labels=labels
            ),
            weighted_recall=recall_score(
                y_true, y_pred, average="weighted", labels=labels
            ),
        )

    if "samples" in average:
        ret.update(
            samples_f1=f1_score(
                y_true, y_pred, average="samples", labels=labels
            ),
            samples_precision=precision_score(
                y_true, y_pred, average="samples", labels=labels
            ),
            samples_recall=recall_score(
                y_true, y_pred, average="samples", labels=labels
            ),
        )

    return ret

In [ ]:
# DIRPATH_LOAD = f'hyper_groupCV/testset_{IDX_CV_TEST}'
DIRPATH_LOAD = dir_base
DIRPATH_LOAD = [x for x in os.listdir(dir_base) if x[:2] == "cv"]
DIRPATH_LOAD.sort(key=lambda item: int(item[2:]))

hyper_para = pd.read_csv(f"{dir_base}/list_hyperparameters.csv", index_col=0)
x_lim_range = [0, epochs * n_minibatch_PI]  # plot only main training parts

df_summary = pd.DataFrame()
for dirL in DIRPATH_LOAD:
    mdl_list = [
        x for x in os.listdir(f"{dir_base}/{dirL}") if x[:5] == "hpara"
    ]
    mdl_list.sort(key=lambda item: int(item[5:]))
    for fn in mdl_list:
        learning_curve = pd.read_csv(
            f"{dir_base}/{dirL}/{fn}/learning_curve.csv", index_col=0
        )
        idx_list = [
            x
            for x in learning_curve.index
            if (x[:5] == "main_") and (int(x.split("_")[1]) >= burn_in)
        ]
        idx = learning_curve["Loss_Target_Validation"].loc[idx_list].idxmin()
        tmp_row = (
            learning_curve.loc[idx, :].rename(f"{dirL}_{fn}").to_frame().T
        )
        tmp_row["cv"] = int(dirL[2:])
        tmp_row["n_epoch"] = int(idx.split("_")[1])
        tmp_row["hpara"] = int(fn[5:])
        tmp_row[hyper_para.columns] = hyper_para.loc[
            tmp_row["hpara"], :
        ].values
        df_summary = pd.concat([df_summary, tmp_row], axis=0)

        # plot learning curves
        best_t = int(idx.split("_")[1]) * n_minibatch_PI + int(
            idx.split("_")[2]
        )

        _ = plt.xlabel("epoch")
        _ = plt.ylabel("Target loss")
        _ = plt.plot(
            learning_curve["Loss_Target_Training"].values, label="Training"
        )
        _ = plt.plot(
            learning_curve["Loss_Target_Validation"].values, label="Validation"
        )
        _ = plt.plot(learning_curve["Loss_Target_Test"].values, label="Test")
        _ = plt.axvline(x=best_t, color="r", ls="--")
        _ = plt.xlim(x_lim_range)
        _ = plt.ylim([0, 1])
        _ = plt.legend()
        _ = plt.savefig(f"{dir_base}/{dirL}/{fn}/LearningCurve_TargetLoss.png")
        # _ = plt.show()
        _ = plt.close()

        _ = plt.xlabel("epoch")
        _ = plt.ylabel("Source loss")
        _ = plt.plot(
            learning_curve["Loss_Source_Training"].values, label="Training"
        )
        _ = plt.plot(learning_curve["Loss_Source_Test"].values, label="Test")
        _ = plt.axvline(x=best_t, color="r", ls="--")
        _ = plt.xlim(x_lim_range)
        _ = plt.ylim([0, 1])
        _ = plt.legend()
        _ = plt.savefig(f"{dir_base}/{dirL}/{fn}/LearningCurve_SourceLoss.png")
        # _ = plt.show()
        _ = plt.close()

        _ = plt.xlabel("epoch")
        _ = plt.ylabel("Target loss")
        _ = plt.plot(
            learning_curve["Loss_Source0_Training"].values, label="Training"
        )
        _ = plt.plot(learning_curve["Loss_Source0_Test"].values, label="Test")
        _ = plt.axvline(x=best_t, color="r", ls="--")
        _ = plt.xlim(x_lim_range)
        _ = plt.ylim([0, 1])
        _ = plt.legend()
        _ = plt.savefig(
            f"{dir_base}/{dirL}/{fn}/LearningCurve_Source0Loss.png"
        )
        # _ = plt.show()
        _ = plt.close()

        # plot predictions
        df_source0_train = pd.read_csv(
            f"{dir_base}/{dirL}/{fn}/best_loss_target_val_source0_train.csv",
            index_col=0,
        )
        df_source0_test = pd.read_csv(
            f"{dir_base}/{dirL}/{fn}/best_loss_target_val_source0_test.csv",
            index_col=0,
        )
        df_source_train = pd.read_csv(
            f"{dir_base}/{dirL}/{fn}/best_loss_target_val_source_train.csv",
            index_col=0,
        )
        df_source_test = pd.read_csv(
            f"{dir_base}/{dirL}/{fn}/best_loss_target_val_source_test.csv",
            index_col=0,
        )
        df_target_train = pd.read_csv(
            f"{dir_base}/{dirL}/{fn}/best_loss_target_val_target_train.csv",
            index_col=0,
        )
        df_target_val = pd.read_csv(
            f"{dir_base}/{dirL}/{fn}/best_loss_target_val_target_val.csv",
            index_col=0,
        )
        df_target_test = pd.read_csv(
            f"{dir_base}/{dirL}/{fn}/best_loss_target_val_target_test.csv",
            index_col=0,
        )
        pd.DataFrame(
            confusion_matrix(
                df_source0_train["y"], df_source0_train["pred"] > 0.5
            )
        ).to_csv(f"{dir_base}/{dirL}/{fn}/confusion_matrix_train.csv")
        pd.DataFrame(
            confusion_matrix(
                df_source0_test["y"], df_source0_test["pred"] > 0.5
            )
        ).to_csv(f"{dir_base}/{dirL}/{fn}/confusion_matrix_test.csv")
        pd.DataFrame(
            classification_metrics(
                df_source0_train["y"].values,
                (df_source0_train["pred"] > 0.5).values,
                average="binary",
            )
        ).to_csv(f"{dir_base}/{dirL}/{fn}/classification_metrics_train.csv")
        pd.DataFrame(
            classification_metrics(
                df_source0_test["y"].values,
                (df_source0_test["pred"] > 0.5).values,
                average="binary",
            )
        ).to_csv(f"{dir_base}/{dirL}/{fn}/classification_metrics_test.csv")

        _ = plt.figure()
        _ = plt.xlabel("Prediction")
        _ = plt.ylabel("Observation")
        _ = plt.scatter(
            df_source_train["pred"],
            df_source_train["y"],
            c="b",
            alpha=0.4,
            label="Train",
        )
        _ = plt.scatter(
            df_source_test["pred"],
            df_source_test["y"],
            c="r",
            alpha=0.4,
            label="Test",
        )
        xy_min = min(df_source_test.min().min(), df_source_train.min().min())
        xy_max = max(df_source_test.max().max(), df_source_train.max().max())
        _ = plt.plot(
            [xy_min, xy_max], [xy_min, xy_max], color="k", ls="--", alpha=0.5
        )
        _ = plt.legend()
        _ = plt.savefig(f"{dir_base}/{dirL}/{fn}/P2O_BestVal_Source.png")
        # _ = plt.show()
        _ = plt.close()

        source_summary = pd.concat(
            [
                pd.DataFrame.from_dict(
                    regression_metrics(
                        df_source_train["y"].values,
                        df_source_train["pred"].values,
                    ),
                    orient="index",
                ),
                pd.DataFrame.from_dict(
                    regression_metrics(
                        df_source_test["y"].values,
                        df_source_test["pred"].values,
                    ),
                    orient="index",
                ),
            ],
            axis=1,
        )
        source_summary.columns = ["train", "test"]
        source_summary.to_csv(
            f"{dir_base}/{dirL}/{fn}/PredSummary_BestVal_Source.csv"
        )

        _ = plt.figure()
        _ = plt.xlabel("Prediction")
        _ = plt.ylabel("Observation")
        _ = plt.scatter(
            df_target_train["pred"],
            df_target_train["y"],
            c="b",
            alpha=0.2,
            label="Train",
        )
        _ = plt.scatter(
            df_target_val["pred"],
            df_target_val["y"],
            c="g",
            alpha=0.3,
            label="Val",
        )
        _ = plt.scatter(
            df_target_test["pred"],
            df_target_test["y"],
            c="r",
            alpha=0.4,
            label="Test",
        )
        xy_min = min(
            df_target_test.min().min(),
            df_target_val.min().min(),
            df_target_train.min().min(),
        )
        xy_max = max(
            df_target_test.max().max(),
            df_target_val.max().max(),
            df_target_train.max().max(),
        )
        _ = plt.plot(
            [xy_min, xy_max], [xy_min, xy_max], color="k", ls="--", alpha=0.5
        )
        _ = plt.legend()
        _ = plt.savefig(f"{dir_base}/{dirL}/{fn}/P2O_BestVal_Target.png")
        # _ = plt.show()
        _ = plt.close()

        target_summary = pd.concat(
            [
                pd.DataFrame.from_dict(
                    regression_metrics(
                        df_target_train["y"].values,
                        df_target_train["pred"].values,
                    ),
                    orient="index",
                ),
                pd.DataFrame.from_dict(
                    regression_metrics(
                        df_target_val["y"].values, df_target_val["pred"].values
                    ),
                    orient="index",
                ),
                pd.DataFrame.from_dict(
                    regression_metrics(
                        df_target_test["y"].values,
                        df_target_test["pred"].values,
                    ),
                    orient="index",
                ),
            ],
            axis=1,
        )
        target_summary.columns = ["train", "val", "test"]
        target_summary.to_csv(
            f"{dir_base}/{dirL}/{fn}/PredSummary_BestVal_Target.csv"
        )

df_summary.sort_values(by="Loss_Target_Validation")
df_summary.groupby("hpara").median().sort_values("Loss_Target_Validation")
df_summary.groupby("hpara").std().sort_values("Loss_Target_Validation")

Plot CV training results

In [ ]:
_ = plt.figure()
_ = df_summary.iloc[:, :9].boxplot()
_ = plt.figure()
_ = df_summary.iloc[:, 4:7].boxplot()

In [ ]:
_ = plt.figure()
_ = sns.boxplot(data=df_summary, x="cv", y="Loss_Target_Validation")
_ = plt.figure()
_ = sns.boxplot(data=df_summary, x="cv", y="Loss_Target_Test")

In [ ]:
_ = plt.figure()
_ = plt.scatter(
    df_summary["Loss_Target_Validation"].values,
    df_summary["Loss_Target_Test"].values,
    alpha=0.5,
)
_ = plt.xlabel("Loss (target-val)")
_ = plt.ylabel("Loss (target-test)")

_ = plt.figure()
_ = plt.scatter(
    df_summary["Loss_Source0_Test"].values,
    df_summary["Loss_Target_Test"].values,
    alpha=0.5,
)
_ = plt.xlabel("Loss (PoLyInfo-test)")
_ = plt.ylabel("Loss (target-test)")

_ = plt.figure()
_ = plt.scatter(
    df_summary["Loss_Source_Test"].values,
    df_summary["Loss_Target_Test"].values,
    alpha=0.5,
)
_ = plt.xlabel("Loss (COSMO-test)")
_ = plt.ylabel("Loss (target-test)")

In [ ]:
_ = plt.figure()
_ = df_summary["Loss_Source0_Training"].hist(bins=50)
_ = plt.figure()
_ = df_summary["Loss_Source0_Test"].hist(bins=50)
_ = plt.figure()
_ = df_summary["Loss_Source_Training"].hist(bins=50)
_ = plt.figure()
_ = df_summary["Loss_Source_Test"].hist(bins=50)
_ = plt.figure()
_ = df_summary["Loss_Target_Training"].hist(bins=50)
_ = plt.figure()
_ = df_summary["Loss_Target_Validation"].hist(bins=50)
_ = plt.figure()
_ = df_summary["Loss_Target_Test"].hist(bins=50)

In [ ]:
tar_prop = "Loss_Source0_Test"
for col, vec_col in df_summary.iloc[:, -6:].items():
    _ = plt.figure()
    _ = plt.scatter(vec_col.values, df_summary[tar_prop].values, alpha=0.5)
    _ = plt.title(col)

In [ ]:
tar_prop = "Loss_Source_Test"
for col, vec_col in df_summary.iloc[:, -6:].items():
    _ = plt.figure()
    _ = plt.scatter(vec_col.values, df_summary[tar_prop].values, alpha=0.5)
    _ = plt.title(col)

In [ ]:
tar_prop = "Loss_Target_Training"
for col, vec_col in df_summary.iloc[:, -6:].items():
    _ = plt.figure()
    _ = plt.scatter(vec_col.values, df_summary[tar_prop].values, alpha=0.5)
    _ = plt.title(col)

In [ ]:
tar_prop = "Loss_Target_Validation"
for col, vec_col in df_summary.iloc[:, -6:].items():
    _ = plt.figure()
    _ = plt.scatter(vec_col.values, df_summary[tar_prop].values, alpha=0.5)
    _ = plt.title(col)

In [ ]:
tar_prop = "Loss_Target_Test"
for col, vec_col in df_summary.iloc[:, -6:].items():
    _ = plt.figure()
    _ = plt.scatter(vec_col.values, df_summary[tar_prop].values, alpha=0.5)
    _ = plt.title(col)

## Final model training

In [ ]:
dir_base2 = "final_models/" + dir_base.split("/")[1]
os.makedirs(dir_base2, exist_ok=True)

poly_group = data_Chi.loc[
    idx_split_t["idx_tr"], COLNAME_SOLUBILITY_POLYMER_PAIR
]

gp_split = GroupShuffleSplit(
    n_splits=n_final_model,
    test_size=test_ratio_final,
    random_state=rng.integers(2**31 - 1),
)
idx_trs_fin, idx_vals_fin = [], []

for idx_tr, idx_val in gp_split.split(
    y_t["y"].loc[idx_split_t["idx_tr"]], groups=poly_group.to_list()
):
    idx_trs_fin.append(
        y_t["y"].loc[idx_split_t["idx_tr"]].iloc[idx_tr].index.values
    )
    idx_vals_fin.append(
        y_t["y"].loc[idx_split_t["idx_tr"]].iloc[idx_val].index.values
    )

In [ ]:
tmp_best_para = (
    df_summary.groupby("hpara")
    .median()
    .sort_values("Loss_Target_Validation")
    .iloc[0, :][["alpha1", "alpha2", "dim_out", "lr"]]
)
alpha1 = tmp_best_para["alpha1"]
alpha2 = tmp_best_para["alpha2"]
dim_out = int(tmp_best_para["dim_out"])
learning_rate = tmp_best_para["lr"]

alpha1, alpha2, dim_out, learning_rate

In [ ]:
for iCV, (idx_tr, idx_val) in enumerate(zip(idx_trs_fin, idx_vals_fin)):
    XT_P_TR = torch.tensor(
        desc_t_s.loc[idx_tr, dname_p].values.astype("float"),
        dtype=torch.float32,
        device=device,
    )
    XT_S_TR = torch.tensor(
        desc_t_s.loc[idx_tr, dname_s].values.astype("float"),
        dtype=torch.float32,
        device=device,
    )
    XT_T_TR = torch.tensor(
        temp_t_s.loc[idx_tr, :].values.astype("float"),
        dtype=torch.float32,
        device=device,
    )
    YT_TR = torch.tensor(
        y_t_s.loc[idx_tr, :].values.astype("float"),
        dtype=torch.float32,
        device=device,
    )

    XT_P_VA = torch.tensor(
        desc_t_s.loc[idx_val, dname_p].values.astype("float"),
        dtype=torch.float32,
        device=device,
    )
    XT_S_VA = torch.tensor(
        desc_t_s.loc[idx_val, dname_s].values.astype("float"),
        dtype=torch.float32,
        device=device,
    )
    XT_T_VA = torch.tensor(
        temp_t_s.loc[idx_val, :].values.astype("float"),
        dtype=torch.float32,
        device=device,
    )
    YT_VA = torch.tensor(
        y_t_s.loc[idx_val, :].values.astype("float"),
        dtype=torch.float32,
        device=device,
    )

    generator_p = ParameterGenerator(
        in_features=dim_in_p,
        out_features=dim_out * 2,
        h_neurons=dict(
            data=lambda x: neuron_vector(x, dim_in_p, dim_out * 2),
            repeat=[n_NNlayer],
        ),
        h_activation_funcs=(nn.Sigmoid(),),
        h_dropouts=(0.0,),
    )

    generator_s = ParameterGenerator(
        in_features=dim_in_s,
        out_features=dim_out * 2,
        h_neurons=dict(
            data=lambda x: neuron_vector(x, dim_in_s, dim_out * 2),
            repeat=[n_NNlayer],
        ),
        h_activation_funcs=(nn.Sigmoid(),),
        h_dropouts=(0.0,),
    )

    for iM, ((paras_p, model_p), (paras_s, model_s)) in enumerate(
        zip(
            generator_p(num=1, factory=SequentialLinear),
            generator_s(num=1, factory=SequentialLinear),
        )
    ):
        dir_save = f"{dir_base2}/model_{iCV}"
        os.makedirs(dir_save, exist_ok=True)

        c_model = Chi_Model(model_p, model_s, dim_out)
        _ = c_model.to(device)

        _ = c_model.train()

        optimizer = optim.Adam(
            c_model.parameters(), lr=learning_rate, amsgrad=True
        )
        scheduler = StepLR(optimizer, step_size=sch_step_size, gamma=sch_gamma)

        learning_curve = pd.DataFrame()

        for t in range(epochs_s):
            # mini-batch of training data
            kf = KFold(
                n_splits=n_minibatch_PI,
                shuffle=True,
                random_state=rng.integers(2**31 - 1),
            )
            idx_mb_s0 = [x for _, x in kf.split(XS0_P_TR)]

            # pre-training with PI
            for tt, ii_s0 in enumerate(idx_mb_s0):
                _ = c_model.train()

                tmp_source0_train = c_model(
                    XS0_P_TR[ii_s0, :], XS0_S_TR[ii_s0, :], XS0_T_TR[ii_s0, :]
                )
                py_source0_train = tmp_source0_train[:, 0:1]
                loss_source0_train = criterion_source0(
                    py_source0_train, YS0_TR[ii_s0, :]
                )

                optimizer.zero_grad()
                loss_source0_train.backward()
                optimizer.step()

                _ = c_model.eval()
                with torch.no_grad():
                    py_source_train = c_model(XS_P_TR, XS_S_TR, XS_T_TR)[
                        :, 1:2
                    ]
                    py_target_train = c_model(XT_P_TR, XT_S_TR, XT_T_TR)[
                        :, 2:3
                    ]
                    loss_source_train = criterion_source(
                        py_source_train, YS_TR
                    )
                    loss_target_train = (
                        criterion_target(py_target_train, YT_TR)
                        * loss_factor_target
                    )
                    loss_train = alpha1 * loss_source0_train + (
                        1.0 - alpha1
                    ) * (
                        alpha2 * loss_source_train
                        + (1.0 - alpha2) * loss_target_train
                    )

                    py_target_val = c_model(XT_P_VA, XT_S_VA, XT_T_VA)[:, 1:2]
                    loss_target_val = (
                        criterion_target(py_target_val, YT_VA)
                        * loss_factor_target
                    )

                    py_source0_test = c_model(XS0_P_TE, XS0_S_TE, XS0_T_TE)[
                        :, 0:1
                    ]
                    py_source_test = c_model(XS_P_TE, XS_S_TE, XS_T_TE)[:, 1:2]
                    py_target_test = c_model(XT_P_TE, XT_S_TE, XT_T_TE)[:, 2:3]
                    loss_source0_test = criterion_source0(
                        py_source0_test, YS0_TE
                    )
                    loss_source_test = criterion_source(py_source_test, YS_TE)
                    loss_target_test = (
                        criterion_target(py_target_test, YT_TE)
                        * loss_factor_target
                    )
                    loss_test = alpha1 * loss_source0_test + (1.0 - alpha1) * (
                        alpha2 * loss_source_test
                        + (1.0 - alpha2) * loss_target_test
                    )

                learning_curve = pd.concat(
                    [
                        learning_curve,
                        pd.Series(
                            {
                                "Loss_Source0_Training": loss_source0_train.item(),
                                "Loss_Source0_Test": loss_source0_test.item(),
                                "Loss_Source_Training": loss_source_train.item(),
                                "Loss_Source_Test": loss_source_test.item(),
                                "Loss_Target_Training": loss_target_train.item(),
                                "Loss_Target_Validation": loss_target_val.item(),
                                "Loss_Target_Test": loss_target_test.item(),
                                "Loss_Training": loss_train.item(),
                                "Loss_Test": loss_test.item(),
                            },
                            name=f"pre_{t}",
                        )
                        .to_frame()
                        .T,
                    ],
                    axis=0,
                )

        # main training
        best_loss_val = np.inf
        for t in range(epochs):
            # mini-batch of training data
            kf = KFold(n_splits=n_minibatch_PI, shuffle=True)
            idx_mb_s0 = [x for _, x in kf.split(XS0_P_TR)]
            idx_mb_s, idx_mb_t = [], []
            for k in range(n_factor_COSMO):
                kf = KFold(n_splits=n_minibatch_COSMO, shuffle=True)
                idx_mb_s += [x for _, x in kf.split(XS_P_TR)]
            for k in range(n_factor_CHI):
                kf = KFold(n_splits=n_minibatch_CHI, shuffle=True)
                idx_mb_t += [x for _, x in kf.split(XT_P_TR)]

            for tt, (ii_s0, ii_s, ii_t) in enumerate(
                zip(idx_mb_s0, idx_mb_s, idx_mb_t)
            ):
                c_model.train()
                if alpha1 > 0:
                    py_source0_train = c_model(
                        XS0_P_TR[ii_s0, :],
                        XS0_S_TR[ii_s0, :],
                        XS0_T_TR[ii_s0, :],
                    )[:, 0:1]
                    loss_source0_train = criterion_source0(
                        py_source0_train, YS0_TR[ii_s0, :]
                    )
                else:
                    loss_source0_train = torch.zeros(1, device=device)

                if (alpha2 > 0) and (alpha1 < 1):
                    if no_COSMO_BN:
                        c_model.eval()
                    else:
                        c_model.train()
                    py_source_train = c_model(
                        XS_P_TR[ii_s, :], XS_S_TR[ii_s, :], XS_T_TR[ii_s, :]
                    )[:, 1:2]
                    loss_source_train = criterion_source(
                        py_source_train, YS_TR[ii_s, :]
                    )
                else:
                    loss_source_train = torch.zeros(1, device=device)

                if (alpha1 < 1) and (alpha2 < 1):
                    if no_target_BN:
                        c_model.eval()
                    else:
                        c_model.train()
                    py_target_train = c_model(
                        XT_P_TR[ii_t, :], XT_S_TR[ii_t, :], XT_T_TR[ii_t, :]
                    )[:, 2:3]
                    loss_target_train = (
                        criterion_target(py_target_train, YT_TR[ii_t, :])
                        * loss_factor_target
                    )
                else:
                    loss_target_train = torch.zeros(1, device=device)

                loss_train = alpha1 * loss_source0_train + (1.0 - alpha1) * (
                    alpha2 * loss_source_train
                    + (1.0 - alpha2) * loss_target_train
                )

                optimizer.zero_grad()
                loss_train.backward()
                optimizer.step()

                c_model.eval()
                with torch.no_grad():
                    py_target_val = c_model(XT_P_VA, XT_S_VA, XT_T_VA)[:, 2:3]
                    loss_target_val = (
                        criterion_target(py_target_val, YT_VA)
                        * loss_factor_target
                    )

                    py_source0_test = c_model(XS0_P_TE, XS0_S_TE, XS0_T_TE)[
                        :, 0:1
                    ]
                    py_source_test = c_model(XS_P_TE, XS_S_TE, XS_T_TE)[:, 1:2]
                    py_target_test = c_model(XT_P_TE, XT_S_TE, XT_T_TE)
                    loss_source0_test = criterion_source0(
                        py_source0_test, YS0_TE
                    )
                    loss_source_test = criterion_source(py_source_test, YS_TE)
                    loss_target_test = (
                        criterion_target(py_target_test[:, 2:3], YT_TE)
                        * loss_factor_target
                    )
                    loss_test = alpha1 * loss_source0_test + (1.0 - alpha1) * (
                        alpha2 * loss_source_test
                        + (1.0 - alpha2) * loss_target_test
                    )

                learning_curve = pd.concat(
                    [
                        learning_curve,
                        pd.Series(
                            {
                                "Loss_Source0_Training": loss_source0_train.item(),
                                "Loss_Source0_Test": loss_source0_test.item(),
                                "Loss_Source_Training": loss_source_train.item(),
                                "Loss_Source_Test": loss_source_test.item(),
                                "Loss_Target_Training": loss_target_train.item(),
                                "Loss_Target_Validation": loss_target_val.item(),
                                "Loss_Target_Test": loss_target_test.item(),
                                "Loss_Training": loss_train.item(),
                                "Loss_Test": loss_test.item(),
                            },
                            name=f"main_{t}_{tt}",
                        )
                        .to_frame()
                        .T,
                    ],
                    axis=0,
                )

                if (t > burn_in) and (loss_target_val < best_loss_val):
                    save_NN(
                        paras_p,
                        paras_s,
                        dim_out,
                        c_model,
                        f"{dir_save}/best_loss_target_val.pt",
                    )

                    with torch.no_grad():
                        py_source0_train = c_model(
                            XS0_P_TR, XS0_S_TR, XS0_T_TR
                        )[:, 0:1]
                        py_source_train = c_model(XS_P_TR, XS_S_TR, XS_T_TR)[
                            :, 1:2
                        ]
                        py_target_train = c_model(XT_P_TR, XT_S_TR, XT_T_TR)[
                            :, 2:3
                        ]
                    pd.concat(
                        [
                            y_s0.loc[idx_split_s0["idx_tr"], :],
                            pd.Series(
                                py_source0_train.to("cpu")
                                .detach()
                                .numpy()
                                .flatten(),
                                index=idx_split_s0["idx_tr"],
                                name="pred",
                            ),
                        ],
                        axis=1,
                    ).to_csv(
                        f"{dir_save}/best_loss_target_val_source0_train.csv"
                    )
                    pd.concat(
                        [
                            y_s.loc[idx_split_s["idx_tr"], :],
                            pd.Series(
                                ys_scaler.inverse_transform(
                                    py_source_train.to("cpu").detach().numpy()
                                ).flatten(),
                                index=idx_split_s["idx_tr"],
                                name="pred",
                            ),
                        ],
                        axis=1,
                    ).to_csv(
                        f"{dir_save}/best_loss_target_val_source_train.csv"
                    )
                    pd.concat(
                        [
                            y_t.loc[idx_tr, :],
                            pd.Series(
                                yt_scaler.inverse_transform(
                                    py_target_train.to("cpu").detach().numpy()
                                ).flatten(),
                                index=idx_tr,
                                name="pred",
                            ),
                        ],
                        axis=1,
                    ).to_csv(
                        f"{dir_save}/best_loss_target_val_target_train.csv"
                    )

                    pd.concat(
                        [
                            y_t.loc[idx_val, :],
                            pd.Series(
                                yt_scaler.inverse_transform(
                                    py_target_val.to("cpu").detach().numpy()
                                ).flatten(),
                                index=idx_val,
                                name="pred",
                            ),
                        ],
                        axis=1,
                    ).to_csv(f"{dir_save}/best_loss_target_val_target_val.csv")

                    pd.concat(
                        [
                            y_s0.loc[idx_split_s0["idx_te"], :],
                            pd.Series(
                                py_source0_test.to("cpu")
                                .detach()
                                .numpy()
                                .flatten(),
                                index=idx_split_s0["idx_te"],
                                name="pred",
                            ),
                        ],
                        axis=1,
                    ).to_csv(
                        f"{dir_save}/best_loss_target_val_source0_test.csv"
                    )
                    pd.concat(
                        [
                            y_s.loc[idx_split_s["idx_te"], :],
                            pd.Series(
                                ys_scaler.inverse_transform(
                                    py_source_test.to("cpu").detach().numpy()
                                ).flatten(),
                                index=idx_split_s["idx_te"],
                                name="pred",
                            ),
                        ],
                        axis=1,
                    ).to_csv(
                        f"{dir_save}/best_loss_target_val_source_test.csv"
                    )
                    pd.concat(
                        [
                            y_t.loc[idx_split_t["idx_te"], :],
                            pd.Series(
                                yt_scaler.inverse_transform(
                                    py_target_test[:, 2:3]
                                    .to("cpu")
                                    .detach()
                                    .numpy()
                                ).flatten(),
                                index=idx_split_t["idx_te"],
                                name="pred",
                            ),
                        ],
                        axis=1,
                    ).to_csv(
                        f"{dir_save}/best_loss_target_val_target_test.csv"
                    )

                    tmp_mat = py_target_test.to("cpu").detach().numpy()
                    tmp_mat[:, 1:2] = ys_scaler.inverse_transform(
                        tmp_mat[:, 1:2]
                    )
                    tmp_mat[:, 2:3] = yt_scaler.inverse_transform(
                        tmp_mat[:, 2:3]
                    )
                    tmp_mat = pd.DataFrame(tmp_mat)
                    tmp_mat.columns = [
                        "Soluble",
                        "Chi_COSMO",
                        "Chi_Exp",
                        "Z_sol",
                        "A_COSMO",
                        "B_COSMO",
                        "A_Exp",
                        "B_Exp",
                    ] + [f"Z_{x}" for x in range(dim_out)]
                    tmp_mat.to_csv(f"{dir_save}/output_target_test.csv")

                    best_loss_val = loss_target_val

            scheduler.step()

        learning_curve.to_csv(f"{dir_save}/learning_curve.csv")

    print(f"Finished model {iCV}")